In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'spark-3.4.0-bin-hadoop3.tgz'


In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
from google.colab import files
import pandas as pd
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

# Show DataFrame
df.show()

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price = spark.sql(("""
    SELECT
      YEAR(date) AS year,
      CONCAT('$', format_number(ROUND(AVG(price), 2), '0,000.00')) AS price
    FROM
      home_sales
    WHERE
      bedrooms = 4
    GROUP BY
      YEAR(date)
    ORDER BY
      year DESC
"""))
avg_price.show()

In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price = spark.sql(("""
    SELECT
      YEAR(date) AS year,
      CONCAT('$', format_number(ROUND(AVG(price), 2), '0,000.00')) AS price
    FROM
      home_sales
    WHERE
      bedrooms = 3
      AND
      bathrooms = 3
    GROUP BY
      YEAR(date)
    ORDER BY
      year DESC
"""))
avg_price.show()

In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price = spark.sql("""
    SELECT
      YEAR(date) AS year,
      CONCAT('$', format_number(ROUND(AVG(price), 2), '0,000.00')) AS price
    FROM
      home_sales
    WHERE
      bedrooms = 3
      AND
      bathrooms = 3
      AND
      floors = 2
      AND
      sqft_living >= 2000
    GROUP BY
      YEAR(date)
    ORDER BY year DESC
""")
avg_price.show()

In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_view = spark.sql("""
SELECT
    view,
    CONCAT('$', format_number(ROUND(AVG(price), 2), '0,000.00')) AS price
FROM
    home_sales
WHERE
    price >= 350000
GROUP BY
    view
ORDER BY
    CAST(view AS DOUBLE) DESC
""")

avg_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.

spark.sql("CACHE TABLE home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

avg_view = spark.sql("""
SELECT
    view,
    CONCAT('$', format_number(ROUND(AVG(price), 2), '0,000.00')) AS price
FROM
    home_sales
WHERE
    price >= 350000
GROUP BY
    view
ORDER BY
    CAST(view AS DOUBLE) DESC
""")

avg_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("homes_parquet")

In [ ]:
# 11. Read the parquet formatted data.
parquet_df=spark.read.parquet('homes_parquet')

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('homes_parquet')


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

avg_view = spark.sql("""
SELECT
    view,
    CONCAT('$', format_number(ROUND(AVG(price), 2), '0,000.00')) AS price
FROM
    homes_parquet
WHERE
    price >= 350000
GROUP BY
    view
ORDER BY
    CAST(view AS DOUBLE) DESC
""")

avg_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

